In [38]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.distributions import Categorical, Normal, Multinomial, Binomial, MultivariateNormal, Beta, constraints
from pyro.distributions.torch import Bernoulli
import pyro.infer as infer
from pyro.infer import TraceEnum_ELBO, Trace_ELBO, config_enumerate
from pyro import poutine
from pyro.poutine.trace_messenger import TraceMessenger
from pyro.poutine.enum_messenger import EnumMessenger
from pyro.poutine.messenger import Messenger

plt.style.use('seaborn-v0_8')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def model():
    p = pyro.param("p", torch.softmax(torch.randn(1, 6), -1).squeeze())
    a = pyro.sample("a", Categorical(p))
    pb = torch.randn(6, )
    print(pb, torch.sigmoid(pb))
    b = pyro.sample("b", Bernoulli(logits=pb[a]))
    c = pyro.sample("c", Normal(b, torch.tensor(1.)))
    return 
pyro.set_rng_seed(0)
torch.manual_seed(0)
pyro.clear_param_store()
model = config_enumerate(model)
trace = poutine.trace(poutine.enum(model, first_available_dim=-2)).get_trace()

tensor([ 0.4033,  0.8380, -0.7193, -0.4033, -0.5966,  0.1820]) tensor([0.5995, 0.6980, 0.3276, 0.4005, 0.3551, 0.5454])


In [3]:
trace.compute_log_prob()

In [4]:
trace.format_shapes().split("\n")

['Trace Shapes:        ',
 ' Param Sites:        ',
 '            p       6',
 'Sample Sites:        ',
 '       a dist       |',
 '        value   6 1 |',
 '     log_prob   6 1 |',
 '       b dist   6 1 |',
 '        value 2 1 1 |',
 '     log_prob 2 6 1 |',
 '       c dist 2 1 1 |',
 '        value 2 1 1 |',
 '     log_prob 2 1 1 |']

In [5]:
trace.nodes["a"]

{'type': 'sample',
 'name': 'a',
 'fn': Categorical(probs: torch.Size([6]), logits: torch.Size([6])),
 'is_observed': False,
 'args': (),
 'kwargs': {},
 'value': tensor([[0],
         [1],
         [2],
         [3],
         [4],
         [5]]),
 'infer': {'enumerate': 'parallel',
  'expand': False,
  '_enumerate_dim': -2,
  '_dim_to_id': {-2: 0}},
 'scale': 1.0,
 'mask': None,
 'cond_indep_stack': (),
 'done': True,
 'stop': False,
 'continuation': None,
 'unscaled_log_prob': tensor([[-0.5232],
         [-2.3576],
         [-4.2429],
         [-1.4957],
         [-3.1487],
         [-3.4627]], grad_fn=<SqueezeBackward1>),
 'log_prob': tensor([[-0.5232],
         [-2.3576],
         [-4.2429],
         [-1.4957],
         [-3.1487],
         [-3.4627]], grad_fn=<SqueezeBackward1>),
 'log_prob_sum': tensor(-15.2308, grad_fn=<SumBackward0>)}

In [6]:
trace.nodes["a"]["log_prob"].exp()

tensor([[0.5926],
        [0.0946],
        [0.0144],
        [0.2241],
        [0.0429],
        [0.0313]], grad_fn=<ExpBackward0>)

In [7]:
trace.nodes["a"]['fn']

Categorical(probs: torch.Size([6]), logits: torch.Size([6]))

In [8]:
trace.nodes["a"]["fn"].logits

tensor([-0.5232, -2.3576, -4.2429, -1.4957, -3.1487, -3.4627],
       grad_fn=<LogBackward0>)

In [9]:
trace.nodes["a"]["fn"].probs

tensor([0.5926, 0.0946, 0.0144, 0.2241, 0.0429, 0.0313],
       grad_fn=<DivBackward0>)

In [10]:
probs = trace.nodes["a"]["fn"].probs

In [11]:
torch.log(probs)

tensor([-0.5232, -2.3576, -4.2429, -1.4957, -3.1487, -3.4627],
       grad_fn=<LogBackward0>)

In [12]:
trace.nodes["b"]

{'type': 'sample',
 'name': 'b',
 'fn': Bernoulli(logits: torch.Size([6, 1])),
 'is_observed': False,
 'args': (),
 'kwargs': {},
 'value': tensor([[[0.]],
 
         [[1.]]]),
 'infer': {'enumerate': 'parallel',
  'expand': False,
  '_enumerate_dim': -3,
  '_dim_to_id': {-3: 1, -2: 0}},
 'scale': 1.0,
 'mask': None,
 'cond_indep_stack': (),
 'done': True,
 'stop': False,
 'continuation': None,
 'unscaled_log_prob': tensor([[[-0.9150],
          [-1.1975],
          [-0.3968],
          [-0.5117],
          [-0.4387],
          [-0.7883]],
 
         [[-0.5117],
          [-0.3595],
          [-1.1161],
          [-0.9150],
          [-1.0353],
          [-0.6063]]]),
 'log_prob': tensor([[[-0.9150],
          [-1.1975],
          [-0.3968],
          [-0.5117],
          [-0.4387],
          [-0.7883]],
 
         [[-0.5117],
          [-0.3595],
          [-1.1161],
          [-0.9150],
          [-1.0353],
          [-0.6063]]]),
 'log_prob_sum': tensor(-8.7918)}

In [13]:
probs = trace.nodes["b"]["fn"].probs
probs

tensor([[0.5995],
        [0.6980],
        [0.3276],
        [0.4005],
        [0.3551],
        [0.5454]])

In [14]:
probs = torch.column_stack((1-probs, probs))

In [15]:
probs

tensor([[0.4005, 0.5995],
        [0.3020, 0.6980],
        [0.6724, 0.3276],
        [0.5995, 0.4005],
        [0.6449, 0.3551],
        [0.4546, 0.5454]])

In [16]:
trace.nodes["b"]["log_prob"]

tensor([[[-0.9150],
         [-1.1975],
         [-0.3968],
         [-0.5117],
         [-0.4387],
         [-0.7883]],

        [[-0.5117],
         [-0.3595],
         [-1.1161],
         [-0.9150],
         [-1.0353],
         [-0.6063]]])

In [17]:
torch.stack([torch.log(torch.select(probs, 1, 0)).unsqueeze(1), torch.log(torch.select(probs, 1, 1)).unsqueeze(1)])

tensor([[[-0.9150],
         [-1.1975],
         [-0.3968],
         [-0.5117],
         [-0.4387],
         [-0.7883]],

        [[-0.5117],
         [-0.3595],
         [-1.1161],
         [-0.9150],
         [-1.0353],
         [-0.6063]]])

In [19]:
trace.nodes["c"]["fn"].loc

tensor([[[0.]],

        [[1.]]])

In [20]:
trace.nodes["c"]["log_prob"]

tensor([[[-1.2859]],

        [[-1.5246]]])

In [34]:
def model():
    p = pyro.param("p", torch.softmax(torch.randn(1, 6), -1).squeeze())
    a = pyro.sample("a", Categorical(p))
    print("Model a: ", a)
    return 

def guide():
    p = pyro.param("p", torch.softmax(torch.randn(1, 6), -1).squeeze())
    a = pyro.sample("a", Categorical(p))
    print("Guide a: ", a)
pyro.set_rng_seed(0)
torch.manual_seed(0)
pyro.clear_param_store()
guide = config_enumerate(guide, default="sequential")
trace = poutine.trace(poutine.enum(guide, first_available_dim=-1)).get_trace()

Guide a:  tensor(0)


In [26]:
trace.compute_log_prob()

In [28]:
trace.format_shapes().split("\n")

['Trace Shapes:  ',
 ' Param Sites:  ',
 '            p 6',
 'Sample Sites:  ',
 '       a dist |',
 '        value |',
 '     log_prob |']

In [37]:
elbo = TraceEnum_ELBO(max_plate_nesting=0)
elbo.loss(model, config_enumerate(guide, "sequential"));

Guide a:  tensor(5)
Model a:  tensor(5)
Guide a:  tensor(4)
Model a:  tensor(4)
Guide a:  tensor(3)
Model a:  tensor(3)
Guide a:  tensor(2)
Model a:  tensor(2)
Guide a:  tensor(1)
Model a:  tensor(1)
Guide a:  tensor(0)
Model a:  tensor(0)


In [44]:
guide = config_enumerate(guide, default="sequential")

pyro.set_rng_seed(0)
torch.manual_seed(0)
pyro.clear_param_store()
with EnumMessenger(first_available_dim=-1) as trace:
    guide()

Guide a:  tensor(0)


In [49]:
def model():
    a = pyro.sample("a", dist.Categorical(torch.ones(4)))
    b = pyro.sample("b", dist.Categorical(torch.ones(3)))
    print("Model a: ", a)
    return 

pyro.set_rng_seed(0)
torch.manual_seed(0)
pyro.clear_param_store()
model = config_enumerate(model)
trace = poutine.trace(poutine.enum(model, first_available_dim=-1)).get_trace()

Model a:  tensor([0, 1, 2, 3])


In [50]:
trace.compute_log_prob()

In [51]:
trace.format_shapes().split("\n")

['Trace Shapes:      ',
 ' Param Sites:      ',
 'Sample Sites:      ',
 '       a dist     |',
 '        value   4 |',
 '     log_prob   4 |',
 '       b dist     |',
 '        value 3 1 |',
 '     log_prob 3 1 |']

In [54]:
pyro.sample("p", dist.Dirichlet(torch.ones(3)))

tensor([0.0729, 0.8067, 0.1204])

In [56]:
pyro.sample("p", dist.Dirichlet(torch.ones(3))).sum()

tensor(1.)